In [1]:
import requests

import pandas as pd
import numpy as np




API_KEY = 'ur API key'


In [2]:
SPORT = 'soccer' 

REGIONS = 'eu' 

MARKETS = 'h2h' 

ODDS_FORMAT = 'decimal' 

DATE_FORMAT = 'iso' 

BET_SIZE = 100

odds_response = requests.get(
    f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds',
    params={
        'api_key': API_KEY,
        'regions': REGIONS,
        'markets': MARKETS,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT,
    }
).json()

In [3]:
odds_response

[{'id': '8f10411a9aef22d51622abe6f01cef05',
  'sport_key': 'soccer_italy_serie_a',
  'sport_title': 'Serie A - Italy',
  'commence_time': '2025-03-31T16:30:00Z',
  'home_team': 'Hellas Verona',
  'away_team': 'Parma',
  'bookmakers': [{'key': 'winamax_fr',
    'title': 'Winamax (FR)',
    'last_update': '2025-03-31T13:52:17Z',
    'markets': [{'key': 'h2h',
      'last_update': '2025-03-31T13:52:17Z',
      'outcomes': [{'name': 'Hellas Verona', 'price': 2.6},
       {'name': 'Parma', 'price': 3.0},
       {'name': 'Draw', 'price': 2.95}]}]},
   {'key': 'winamax_de',
    'title': 'Winamax (DE)',
    'last_update': '2025-03-31T13:52:17Z',
    'markets': [{'key': 'h2h',
      'last_update': '2025-03-31T13:52:17Z',
      'outcomes': [{'name': 'Hellas Verona', 'price': 2.6},
       {'name': 'Parma', 'price': 3.0},
       {'name': 'Draw', 'price': 2.95}]}]},
   {'key': 'onexbet',
    'title': '1xBet',
    'last_update': '2025-03-31T13:52:13Z',
    'markets': [{'key': 'h2h',
      'last_upda

In [4]:
def format_odds(odds):
    # Join odds with '/' in European decimal format
    formatted_odds = '/'.join([f"{odd:.2f}" for odd in odds])
    return formatted_odds


def extract_football_odds(odds_response):
    selected_bookmakers = ["Bet365", "Bwin", "Tipico", "Unibet"]
    for event in odds_response:
        home_team = event.get("home_team")
        away_team = event.get("away_team")
        league = event.get("sport_title")
        print(f"\nMatch: {home_team} vs {away_team} - League: {league}")

        best_odds = {'1': (None, 0), 'X': (None, 0), '2': (None, 0)}

        for bookmaker in event.get("bookmakers", []):
            bookie_name = bookmaker.get("title", "")
            # Ensure case-insensitive comparison to avoid mismatch
            if bookie_name.lower() not in [b.lower() for b in selected_bookmakers]:
                continue

            for market in bookmaker.get("markets", []):
                if market.get("key") == "h2h":
                    outcomes = market.get("outcomes", [])
                    if len(outcomes) == 3:
                        odds_list = []
                        for i, outcome in enumerate(["1", "X", "2"]):
                            price = outcomes[i].get("price", 0)
                            odds_list.append(price)
                            if price > best_odds[outcome][1]:
                                best_odds[outcome] = (bookie_name, price)

                        # Format the odds as requested
                        formatted_odds = format_odds(odds_list)
                        print(f"Odds: {formatted_odds}")

        print("Best Odds:")
        for outcome, (bookie, price) in best_odds.items():
            formatted_price = f"{price:.2f}"
            print(f"Outcome {outcome}: {bookie} - {formatted_price}")


# Call the function with the fetched data
extract_football_odds(odds_response)



Match: Hellas Verona vs Parma - League: Serie A - Italy
Odds: 2.55/3.00/3.00
Odds: 2.63/3.10/3.05
Best Odds:
Outcome 1: Unibet - 2.63
Outcome X: Unibet - 3.10
Outcome 2: Unibet - 3.05

Match: Eyüpspor vs Basaksehir - League: Turkey Super League
Odds: 2.55/2.90/3.00
Odds: 2.60/2.75/3.05
Best Odds:
Outcome 1: Unibet - 2.60
Outcome X: Tipico - 2.90
Outcome 2: Unibet - 3.05

Match: FC Copenhagen vs Randers FC - League: Denmark Superliga
Odds: 1.70/4.80/3.90
Best Odds:
Outcome 1: Unibet - 1.70
Outcome X: Unibet - 4.80
Outcome 2: Unibet - 3.90

Match: IFK Värnamo vs IK Sirius - League: Allsvenskan - Sweden
Odds: 2.33/3.10/3.30
Odds: 2.30/2.95/3.20
Best Odds:
Outcome 1: Unibet - 2.33
Outcome X: Unibet - 3.10
Outcome 2: Unibet - 3.30

Match: IK Oddevold vs Umeå FC - League: Superettan - Sweden
Odds: 1.60/5.10/3.90
Odds: 1.50/5.20/4.00
Best Odds:
Outcome 1: Unibet - 1.60
Outcome X: Tipico - 5.20
Outcome 2: Tipico - 4.00

Match: Zagłębie Lubin vs Raków Częstochowa - League: Ekstraklasa - Poland

In [5]:
BOOKMAKER_INDEX = 0
NAME_INDEX = 1
ODDS_INDEX = 2
FIRST = 0

class Event:
    def __init__(self, data):
        self.data = data
        self.sport_key = data['sport_key']
        self.id = data['id']

    def find_best_odds(self):
        # number of possible outcomes for a sporting event
        num_outcomes = len(self.data['bookmakers'][FIRST]['markets'][FIRST]['outcomes'])
        self.num_outcomes = num_outcomes

        # finding the best odds for each outcome in each event
        best_odds = [[None, None, float('-inf')] for _ in range(num_outcomes)]
        # [Bookmaker, Name, Price]

        # Track used bookmakers separately for each outcome
        used_bookmakers = {"1": set(), "X": set(), "2": set()}
        bookmakers = self.data.get('bookmakers', [])
        for bookmaker in bookmakers:
            bookie_name = bookmaker.get('title', '')

            for outcome in range(num_outcomes):
                try:
                    outcome_name = ["1", "X", "2"][outcome]
                    # Skip if this bookmaker is already used for this outcome
                    if bookie_name in used_bookmakers[outcome_name]:
                        continue

                    bookmaker_odds = float(bookmaker['markets'][FIRST]['outcomes'][outcome]['price'])
                    current_best_odds = best_odds[outcome][ODDS_INDEX]

                    if bookmaker_odds > current_best_odds:
                        best_odds[outcome][BOOKMAKER_INDEX] = bookmaker['title']
                        best_odds[outcome][NAME_INDEX] = bookmaker['markets'][FIRST]['outcomes'][outcome]['name']
                        best_odds[outcome][ODDS_INDEX] = bookmaker_odds
                        # Mark this bookmaker as used for this specific outcome
                        used_bookmakers[outcome_name].add(bookie_name)

                except (KeyError, IndexError, ValueError):
                    continue

        self.best_odds = best_odds
        return best_odds

    def arbitrage(self):
        total_arbitrage_percentage = 0
        for odds in self.best_odds:
            total_arbitrage_percentage += (1.0 / odds[ODDS_INDEX])

        self.total_arbitrage_percentage = total_arbitrage_percentage
        self.expected_earnings = (BET_SIZE / total_arbitrage_percentage) - BET_SIZE

        if total_arbitrage_percentage < 1:
            return True
        return False

    def calculate_arbitrage_bets(self):
        bet_amounts = []
        for outcome in range(self.num_outcomes):
            individual_arbitrage_percentage = 1 / self.best_odds[outcome][ODDS_INDEX]
            bet_amount = (BET_SIZE * individual_arbitrage_percentage) / self.total_arbitrage_percentage
            bet_amounts.append(round(bet_amount, 2))

        self.bet_amounts = bet_amounts
        return bet_amounts


In [6]:
events = []
for data in odds_response:
    events.append(Event(data))
    # print(data)
    # print()
    
arbitrage_events = []
for event in events:
    best_odds = event.find_best_odds()
    if event.arbitrage():
        arbitrage_events.append(event)
        
for event in arbitrage_events:
    event.calculate_arbitrage_bets()
    

In [7]:
print(data)

{'id': '0bcb8a7dca0bbffda3c6210e3d555480', 'sport_key': 'soccer_spain_segunda_division', 'sport_title': 'La Liga 2 - Spain', 'commence_time': '2025-03-31T18:30:00Z', 'home_team': 'Cádiz CF', 'away_team': 'SD Eibar', 'bookmakers': [{'key': 'betsson', 'title': 'Betsson', 'last_update': '2025-03-31T13:52:34Z', 'markets': [{'key': 'h2h', 'last_update': '2025-03-31T13:52:34Z', 'outcomes': [{'name': 'Cádiz CF', 'price': 2.25}, {'name': 'SD Eibar', 'price': 3.55}, {'name': 'Draw', 'price': 2.9}]}]}, {'key': 'onexbet', 'title': '1xBet', 'last_update': '2025-03-31T13:52:33Z', 'markets': [{'key': 'h2h', 'last_update': '2025-03-31T13:52:33Z', 'outcomes': [{'name': 'Cádiz CF', 'price': 2.33}, {'name': 'SD Eibar', 'price': 3.66}, {'name': 'Draw', 'price': 2.86}]}]}, {'key': 'nordicbet', 'title': 'Nordic Bet', 'last_update': '2025-03-31T13:52:34Z', 'markets': [{'key': 'h2h', 'last_update': '2025-03-31T13:52:34Z', 'outcomes': [{'name': 'Cádiz CF', 'price': 2.25}, {'name': 'SD Eibar', 'price': 3.55}, 

In [8]:
MAX_OUTCOMES = max([event.num_outcomes for event in arbitrage_events])
ARBITRAGE_EVENTS_COUNT = len(arbitrage_events)

my_columns = ['ID', 'Sport Key', 'Expected Earnings'] + list(np.array([[f'Bookmaker #{outcome}', f'Name #{outcome}', f'Odds #{outcome}', f'Amount to Buy #{outcome}'] for outcome in range(1, MAX_OUTCOMES + 1)]).flatten())
dataframe = pd.DataFrame(columns=my_columns)

In [9]:
for event in arbitrage_events:
    # print(event.best_odds)
    row = []
    row.append(event.id)
    row.append(event.sport_key)
    row.append(round(event.expected_earnings, 2))
    for index, outcome in enumerate(event.best_odds):
        row.append(outcome[BOOKMAKER_INDEX])
        row.append(outcome[NAME_INDEX])
        row.append(outcome[ODDS_INDEX])
        row.append(event.bet_amounts[index])
    while len(row) < len(dataframe.columns):
        row.append('N/A')
    dataframe.loc[len(dataframe.index)] = row

In [10]:
dataframe.head()

,ID,Sport Key,Expected Earnings,Bookmaker #1,Name #1,Odds #1,Amount to Buy #1,Bookmaker #2,Name #2,Odds #2,Amount to Buy #2,Bookmaker #3,Name #3,Odds #3,Amount to Buy #3
0,8f10411a9aef22d51622abe6f01cef05,soccer_italy_serie_a,0.93,1xBet,Hellas Verona,2.81,35.92,Matchbook,Parma,3.15,32.04,Betfair,Draw,3.15,32.04
1,31f4e2a2890c200cbe1c89512425c04a,soccer_sweden_allsvenskan,0.66,Coolbet,IFK Värnamo,2.50,40.27,Matchbook,IK Sirius,3.25,30.97,Betfair,Draw,3.50,28.76
